<a href="https://colab.research.google.com/github/mengjie514/twitter_works/blob/master/PSIII_IV_(demo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Data Collection**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/PSIV.SP/Raw_Data")
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
import json, sched, time, csv

import tweepy
from tweepy import OAuthHandler

# Twitter API
s = sched.scheduler(time.time, time.sleep)
consumer_key = '' 
consumer_secret = ''
access_token_key = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tweepy.API(auth)

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
candidates= [#"SenAngusKing", #"SenShelby", #"TTuberville", #"lisamurkowski", #"DanSullivan_AK", #"SenatorSinema", #"CaptMarkKelly", #"JohnBoozman", #"SenTomCotton", #"SenFeinstein", #"AlexPadilla4CA", #"MichaelBennet", 
             #"Hickenlooper", #"SenBlumenthal", #"ChrisMurphyCT", #"SenatorCarper", #"ChrisCoons", #"marcorubio", #"SenRickScott", #"ossoff", #"ReverendWarnock", #"brianschatz", #"maziehirono", #"crapoforsenate", 
             #"SenatorRisch", #"SenatorDurbin", #"SenDuckworth", #"ToddYoungIN", #"SenatorBraun", #"ChuckGrassley", #"SenJoniErnst", #"JerryMoran", #"RogerMarshallMD", #"LeaderMcConnell", #"RandPaul", #"BillCassidy", 
             #"SenJohnKennedy", #"SenatorCollins", #"SenAngusKing", #"SenatorCardin", #"ChrisVanHollen", #"SenWarren", #"SenMarkey", #"SenStabenow", #"SenGaryPeters", #"amyklobuchar", #"SenTinaSmith", #"SenatorWicker", 
             #"SenHydeSmith", #"RoyBlunt", #"HawleyMO", #"SenatorTester", #"SteveDaines", #"SenatorFischer", #"SenSasse", #"SenCortezMasto", #"RosenforNevada", #"SenatorShaheen", #"SenatorHassan", #"SenatorMenendez", #"CoryBooker", #"MartinHeinrich",
             #"SenatorLujan", #"SenSchumer", #"SenGillibrand", #"SenatorBurr", #"SenThomTillis", #"SenJohnHoeven", #"SenKevinCramer", #"SenSherrodBrown", #"senrobportman", #"JimInhofe", #"SenatorLankford", #"RonWyden", #"SenJeffMerkley", #"SenBobCasey",
             #"SenToomey", #"SenJackReed", #"SenWhitehouse", #"LindseyGrahamSC", #"SenatorTimScott", #"SenJohnThune", #"SenatorRounds", #"MarshaBlackburn", #"BillHagertyTN", #"JohnCornyn", #"tedcruz", #"SenMikeLee", #"MittRomney", #"SenatorLeahy", #"BernieSanders", #"MarkWarner", #"timkaine",
             #"PattyMurray", #"SenatorCantwell", #"Sen_JoeManchin", #"SenCapito", #"SenRonJohnson", #"SenatorBaldwin", #"barrassoforwyo",#"SenLummis"
]

for person in candidates:
    saveFile = open(person+".json",'a')    
    count = 0
   
for tweet in tweepy.Cursor(api.user_timeline,
                                id=person,
                                tweet_mode='extended',
                                wait_on_rate_limit = True,
                                wait_on_rate_limit_notify = True).items():
        saveFile.write(json.dumps(tweet._json))
        saveFile.write("\n")
        count += 1
        print(person+" "+str(count ))
   
saveFile.close

print(person+" finished")

In [ ]:
data = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
      for line in json_file:
        try:
            tweet = json.loads(line)
            if tweet['full_text'].find('RT @')!=-1:
            # drop retweeted tweet
                continue
            data.append(tweet)
        except:
            continue

with open('merge_json.json','w') as fp:
    json.dump(data,fp)

tweets_data_path=('merge_json.json')
tweets_data = []
tweets_file = open(tweets_data_path,'r')
tweets_data = json.load(tweets_file)
type(tweets_data)

# convert json to df
All_df = pd.DataFrame(tweets_data)

# select date in certain period
All_df['created_at'] = pd.to_datetime(All_df.created_at).dt.date
All_df['created_at'] = pd.to_datetime(All_df.created_at, errors='coerce')
#All_df = All_df.loc[(All_df['created_at'] >= '2018-10-02') & (All_df['created_at'] <= '2018-12-30')]

# language in English 
All_df = All_df.loc[All_df['lang'] == 'en']

# remove RT and reply tweets
All_df = All_df[All_df['in_reply_to_screen_name'].isnull()]

# extract elements from dataframe 
col_to_keep = ['created_at', 'id', 'full_text', 'retweet_count','favorite_count', 'user', 'lang']
All_df = All_df[col_to_keep]

add_followers = []
add_friends = []
add_screename = []
add_userid = []
add_userfa = []

for index, row in All_df.iterrows():
    fo = (row['user']['followers_count'])
    fr = (row['user']['friends_count'])
    sc = (row['user']['screen_name'])
    ud = (row['user']['id'])
    uf = (row['user']['favourites_count'])
    
    add_followers.append(fo)
    add_friends.append(fr)
    add_screename.append(sc)
    add_userid.append(ud)
    add_userfa.append(uf)
   
All_df['followers_count'] = add_followers
All_df['friends_count'] = add_friends
All_df['screen_name'] = add_screename
All_df['user_id'] = add_userid
All_df['favourites_count_user'] = add_userfa

del All_df['user']
del All_df['lang']

#All_df.to_csv('PSIV_SP_Raw.csv')

**2. Tweets Pre-processing**

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/PSIII")
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

df = pd.read_csv('Emo_Full.csv', encoding ='unicode_escape')
df = All_df[['created_at', 'full_text', 'retweet_count', 'favorite_count', 'followers_count', 'friends_count', 'screen_name']]

# text Cleaning
import string
from html.parser import HTMLParser
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer, word_tokenize

# hashtags check
hash_list = df.full_text.str.extractall(
    r'(\#\w+)'
).reset_index().drop_duplicates(['level_0', 0])[0].value_counts()

split_adhere_dic = {
    "weve": "we have", "Weve": "We have", "youve": "you have", "Youve": "You have", "theyve": "they have", "Theyve": "They have", "Ive": "I have", "youre": "you are", "Youre": "You are", "theyre": "they are", "Theyre": "They are",
    "Im": "I am", "Ill": "I will", "Id": "I would", "hes": "he's", "Hes": "He's", "shes": "she's", "Shes": "She's", "thats": "that is", "Thats": "That is", "thatd": "that would", "Thatd": "That would", "theres": "there is", "Theres": "There is", 
    "thered": "there would", "Thered": "There would", "heres": "here is", "Heres": "Here is",
    # negation cues
    "isn't": "is not", "isnt": "is not", "ain't": "are not", "aint": "are not", "aren't": "are not", "arent": "are not", "weren't": "were not", "werent": "were not", "wasn't": "was not", "wasnt": " was not", 
    "cannot": "can not", "can't": " can not", "cant": " can not", "couldn't": "could not", "couldnt": " could not", "wouldn't": "would not", "wouldnt": "would not", "don't": "do not", "dont": "do not", "doesn't": "does not", "doesnt": "does not",
    "didn't": "did not", "didnt": "did not", "haven't": "have not", "havent": "have not", "havnt": "have not", "hasn't": "has not", "hasnt": "has not", "hadn't": "had not", "hadnt": "had not", "shouldn't": "should not", "shouldnt": "should not", 
    "won't": "will not", "wont": "will not", "mightn't": "might not", "mightnt": "might not", "mustn't": "must not", "mustnt": "must not", "needn't": "need not", "neednt": "need not"
    }

pat1 = r'@[\w_]+'
pat2 = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
pat3 = r'#'
pat3b = r'#.'
pat4 = r'www.[^ ]+'
pat5 = r'\n'
pat6= r'\xa0'
pat7= r'AT_USER'
pat8 = r'pic.twitter.com/(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
pat9 = r'.twitter.com/(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
combined_pat = r'|'.join((pat1, pat2, pat3, pat3b, pat4, pat5, pat6, pat7, pat8, pat9))

split_pattern = re.compile(r'\b(' + '|'.join(split_adhere_dic.keys()) + r')\b')

def tweet_cleaner(demo):
    soup = BeautifulSoup(demo, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    splitted = re.sub(r"([a-z\.!?])([A-Z])", r"\1 \2", stripped) #split joined words
    split_handled = split_pattern.sub(lambda x: split_adhere_dic[x.group()], splitted)

    return split_handled

df['new_clean_text'] = [tweet_cleaner(t) for t in df.full_text]

# remove stopwords/punctuations/numbers
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# add words that aren't in the NLTK stopwords list and are frequently used in MFT
new_stopwords = ['could', 'would', 'day', 'days', 'month', 'months', 'year', 'years', 
                 'say', 'says', 'said', 'saying', 'gotta', 'wanna', 'etc'] 
new_stopwords_list = stop_words.union(new_stopwords)

# negation Scope Detection for Twitter Sentiment Analysis (Reitan et al, 2015)
not_stopwords = {'no', 'not', 'nor', 'none', 'neither', 'never', 'nothing'}
final_stop_words = set([word for word in new_stopwords_list if word not in not_stopwords])

table = str.maketrans('', '', string.punctuation)

def text_process(deep):
    words = re.split(r'\W+', deep)
    nopunc = [w.translate(table) for w in words]
    nostop =  ' '.join([word for word in nopunc if word.lower() not in final_stop_words])
    nonum = ' '.join([word for word in str(nostop).split() if not word.isdigit()])
    return nonum

df['new_clean_text_deep'] = df.apply(lambda row: text_process(row.new_clean_text), axis=1)

# stemming words
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)
df['new_clean_text_deep_stem'] = df['new_clean_text_deep'].apply(stem_sentences)

We normalized tweets to:

*   convert all alphabets to lowercase to ensure consistency in the text analysis
*   remove noisy symbols (e.g., http://, @user), unnecessary digits, punctuation marks, and stop words (e.g., “a”, “the”) that provide the least value in analysis
*   replace any words containing redundant letters (e.g., “suppoort”) with the correct spellings (e.g., “support”)
*   stem the words (e.g., transferring the inflectional form of “abandoned” to the root form “abandon”) and tokenize the remaining text into the usual format

In [ ]:
df[['full_text', 'new_clean_text_deep_stem', 'screen_name', 'retweet_count', 'favorite_count']].tail(3)

,full_text,new_clean_text_deep_stem,screen_name,retweet_count,favorite_count
14584,"Great to visit with @CBPWestTexas today in El Paso  where the legend began.' We all owe a debt of gratitude to these gents for the sacrifices they make to uphold our laws, secure our borders, and protect our safety. https://t.co/PMQqSHOLi6",great visit today el paso legend began owe debt gratitud gent sacrific make uphold law secur border protect safeti,GOPLeader,118,291
14585,"Thanks to the relentless work of @POTUS and Republicans in Congress, TAXES ARE GOING DOWN AND THE WALL IS GOING UP! #MAGA ?? https://t.co/ZJUrCChtkk",thank relentless work republican congress tax go wall go maga,GOPLeader,4935,13519
14586,"Few things are more fundamental to a nation than a protected border. Proud to introduce the Build the Wall, Enforce the Law Act. #MAGA ?? https://t.co/1nLHgwUvoV",thing fundament nation protect border proud introduc build wall enforc law act maga,GOPLeader,2122,5059
